In [ ]:
using ModelingToolkit, OrdinaryDiffEq

# 定义变量
@variables t
@variables V₁(t) V₂(t) V₃(t) I(t)
# @variables V₁(t) [irreducible = true] V₂(t) [irreducible = true] V₃(t) [irreducible = true] I(t) [irreducible = true]

# 定义微分
D = Differential(t)

# 设置参数
R = 1.0
C = 1.0
V = 1.0

# 输入方程
rc_eqs = [
    V₁ - V₃ ~ V
    V₁ - V₂ ~ I * R
    D(V₂) ~ I / C
    V₃ ~ 0
]

# 构建系统
@named rc_model = ODESystem(rc_eqs, t)
rc_model |> display
rc_model |> states |> display
# 系统化简
sys = structural_simplify(rc_model)

In [ ]:
sys |> println

In [ ]:
sys |> states |> display

In [ ]:
# 设置初值
u0 = [V₂ => 0.0]
# 求解时间范围
tspan = (0.0, 10.0)
# 构建问题并求解
prob = ODAEProblem(sys, u0, tspan)
sol = solve(prob, Tsit5());

In [ ]:
# 查看V₂的变化
sol[V₂]

In [9]:
using ModelingToolkit, OrdinaryDiffEq
# 定义独立时间变量
@variables t
# 器件端口作为连接点
@connector function Pin(; name)
    sts = @variables v(t) = 1.0 i(t) = 1.0 [connect = Flow]
    return ODESystem(Equation[], t, sts, []; name=name)
end
# 地，地的端口名字改为g，不是port
function Ground(; name)
    @named g = Pin()
    eqs = [g.v ~ 0; g.i ~ 0]
    return compose(ODESystem(eqs, t, [], []; name=name), g)
end
# 电阻元件
function Resistor(; name, R=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters R = R
    eqs = [
        p.v - n.v ~ p.i * R
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, [], ps; name=name), p, n)
end
# 电容元件，因为不能对表达式进行微分，所以再引入一个电容的电压差变量v
function Capacitor(; name, C=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters C = C
    sts = @variables begin
        v(t) = 1.0
    end
    D = Differential(t)
    eqs = [
        v ~ p.v - n.v
        D(v) ~ p.i / C
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, sts, ps; name=name), p, n)
end
# 电压源
function ConstantVoltage(; name, V=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters V = V
    eqs = [
        V ~ p.v - n.v
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, [], ps; name=name), p, n)
end
# 定义组件
R = 1.0
C = 1.0
V = 1.0
@named resistor = Resistor(R=R) # @name可以视为给定义的组件起了个名字，Resistor(R=R)返回的组件名字就叫resistor
@named capacitor = Capacitor(C=C)
@named source = ConstantVoltage(V=V)
@named ground = Ground()
# 构建连接关系
rc_eqs = [
    connect(source.p, resistor.p)
    connect(resistor.n, capacitor.p)
    connect(capacitor.n, source.n, ground.g)
]
@named _rc_model = ODESystem(rc_eqs, t) #连接关系也需要放到ODESystem中。
# 组件与组件连接关系一起构建系统
@named rc_model = compose(_rc_model, [resistor, capacitor, source, ground])
equations(rc_model) |> display # 查看方程 |> 
# 系统化简
sys = structural_simplify(rc_model)
equations(sys)  |> display # 查看方程
# 定义初值
u0 = [capacitor.v => 0.0]
# 求解
prob = ODAEProblem(sys, u0, (0, 10.0))
sol = solve(prob, Tsit5(),saveat=0.1)
# 查看电容电压变化
sol(1.35234)

12-element Vector{Equation}:
 connect(source.p, resistor.p)
 connect(resistor.n, capacitor.p)
 connect(capacitor.n, source.n, ground.g)
 -resistor₊n₊v(t) + resistor₊p₊v(t) ~ resistor₊R*resistor₊p₊i(t)
 0 ~ resistor₊p₊i(t) + resistor₊n₊i(t)
 capacitor₊v(t) ~ capacitor₊p₊v(t) - capacitor₊n₊v(t)
 Differential(t)(capacitor₊v(t)) ~ capacitor₊p₊i(t) / capacitor₊C
 0 ~ capacitor₊p₊i(t) + capacitor₊n₊i(t)
 source₊V ~ -source₊n₊v(t) + source₊p₊v(t)
 0 ~ source₊p₊i(t) + source₊n₊i(t)
 ground₊g₊v(t) ~ 0
 ground₊g₊i(t) ~ 0

1-element Vector{Equation}:
 Differential(t)(capacitor₊v(t)) ~ capacitor₊p₊i(t) / capacitor₊C

1-element Vector{Float64}:
 0.7410434348344965

In [6]:
states(sys)

1-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 capacitor₊v(t)

In [8]:
sol[capacitor.p.i]

11-element Vector{Float64}:
 1.0
 0.3678793409002571
 0.13533589030087978
 0.04978186943343821
 0.018311872036761656
 0.006734628215198324
 0.002473203580435235
 0.0009102635543620785
 0.0003409077379603609
 0.00011951081106220673
 5.0071900264403446e-5

In [ ]:
sys.resistor.p.v

In [1]:
# 最新版MTK
using ModelingToolkit, OrdinaryDiffEq
@variables t
@connector Pin begin
    v(t)
    i(t), [connect = Flow]
end
@mtkmodel Ground begin
    @components begin
        g = Pin()
    end
    @equations begin
        g.v ~ 0
    end
end
@mtkmodel OnePort begin
    @components begin
        p = Pin()
        n = Pin()
    end
    @variables begin
        v(t)
        i(t)
    end
    @equations begin
        v ~ p.v - n.v
        0 ~ p.i + n.i
        i ~ p.i
    end
end
@mtkmodel Resistor begin
    @extend OnePort()
    @parameters begin
        R = 1.0 # Sets the default resistance
    end
    @equations begin
        v ~ i * R
    end
end

D = Differential(t)

@mtkmodel Capacitor begin
    @extend OnePort()
    @parameters begin
        C = 1.0
    end
    @equations begin
        D(v) ~ i / C
    end
end

@mtkmodel ConstantVoltage begin
    @extend OnePort()
    @parameters begin
        V = 1.0
    end
    @equations begin
        V ~ v
    end
end

@mtkmodel RCModel begin
    @components begin
        resistor = Resistor(R=1.0)
        capacitor = Capacitor(C=1.0)
        source = ConstantVoltage(V=1.0)
        ground = Ground()
    end
    @equations begin
        connect(source.p, resistor.p)
        connect(resistor.n, capacitor.p)
        connect(capacitor.n, source.n, ground.g)
    end
end

@mtkbuild rc_model = RCModel(resistor.R=2.0)

u0 = [
    rc_model.capacitor.v => 0.0
]
prob = ODEProblem(rc_model, u0, (0, 10.0))
sol = solve(prob, Tsit5())
sol[rc_model.capacitor.p.v]

15-element Vector{Float64}:
 0.0
 4.999875002083307e-5
 0.000549848777725354
 0.005534627202823254
 0.05304729754059866
 0.16870857122402544
 0.3232210676188638
 0.47918255576870894
 0.6305653418370613
 0.7574167496079482
 0.854764329162927
 0.9210274311861326
 0.9615821238825505
 0.9835167746379931
 0.9932587425735419

In [ ]:
rc_model.capacitor

In [ ]:
rc_model.capacitor.v

In [2]:
ConstantVoltage |> propertynames

(:f, :structure, :isconnector)

In [3]:
ConstantVoltage |> typeof

ModelingToolkit.Model{typeof(__ConstantVoltage__), Dict{Symbol, Any}}

In [11]:
ConstantVoltage.structure

Dict{Symbol, Any} with 5 entries:
  :kwargs               => Dict{Symbol, Any}(:V=>1.0)
  :independent_variable => t
  :extend               => Any[[:p, :n, :v, :i], Symbol("#mtkmodel__anonymous__…
  :parameters           => Dict{Symbol, Dict{Symbol, Any}}(:V=>Dict(:default=>1…
  :equations            => Any["V ~ v"]

In [18]:
ConstantVoltage.f

__ConstantVoltage__ (generic function with 1 method)

LoadError: LoadError: MethodError: no method matching var"@component"(::LineNumberNode, ::Module)

Closest candidates are:
  var"@component"(::LineNumberNode, ::Module, !Matched::Any)
   @ ModelingToolkit C:\Users\52243\.julia\packages\ModelingToolkit\Gpzyo\src\systems\abstractsystem.jl:1307

in expression starting at d:\PersonalFile\Develop\Share\4. 组件化.ipynb:1